In [ ]:
import findspark
import os
import pandas as pd
import psycopg2
import requests


from database_functions import (
    get_config,
    insert_values, 
    general_query)
from datetime import datetime


findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [ ]:
#GLOBALS
TABLE_NAME_TARGET='crypto_timeseries'

CONFIG_DB = get_config(filename="config.ini", section="crypto")
CONFIG_API = get_config(filename="config.ini", section="api")
CONFIG_SPARK= get_config(filename="config.ini", section="apache-spark")

API_URL="https://rest.coincap.io/v3/assets?apiKey={api_key}"
HEADER_API = {
            "Content-Type":"application/json",
            "Accept-Encoding":"deflate" 
        }

SPARK_SESSION_NAME="main_etl_pyspark"
POSTRESQL_JARFILE=CONFIG_SPARK['postresql_jar']
COLUMN_DATATYPES = {
            "timestamp":            "timestamp",
            "id":                   "string",
            "rank":                 "int",
            "symbol":               "string",
            "name":                 "string",
            "supply":               "double",
            "maxsupply":            "double",
            "marketcapusd":         "double",
            "volumeusd24hr":        "double",
            "priceusd":             "double",
            "changepercent24hr":    "double",
            "vwap24hr":             "double",
            "explorer":             "string"
            }


In [ ]:
#initiate spark session
print(f"\nInit Spark session..")
spark = SparkSession \
    .builder \
    .appName(SPARK_SESSION_NAME) \
    .config("spark.jars", POSTRESQL_JARFILE) \
    .getOrCreate()

print(f"\nSpark is running at: \n{spark._jsc.sc().uiWebUrl().get()}")

In [ ]:
##----> ETRACT <----- ##
print(f"\nEXTRACT from API query..")
#api query
response = requests.get(url=API_URL.format(api_key=CONFIG_API['api_key']), 
                        headers=HEADER_API)


In [ ]:
##----> TRANSFORM <----- ##
print(f"\nTRANSFORM..")
#api json response to pandas Dataframe
responseData=response.json()
df = pd.json_normalize(data=responseData, record_path='data')

#insert timestamp
current_timestamp = datetime.now()
current_timestamp.strftime('%d-%m-%Y %H:%M:%S')
df['timestamp'] = [current_timestamp]*df.shape[0]

#rename columns to lowercase
rename_cols_dict={c:c.lower() for c in df.columns.tolist()}
df.rename(columns=rename_cols_dict, inplace=True)

#drop columns with tokens*
df = df.loc[:, ~df.columns.str.startswith("tokens.")]

#pandas Dataframe --> spark DataFrame
dfs = spark.createDataFrame(df)

#assure expected columns data types
for column_name, data_type in COLUMN_DATATYPES.items():
    dfs = dfs\
        .withColumn(column_name, col(column_name).cast(data_type))

In [ ]:
##----> LOAD <----- ##
print(f"\nLOAD..")
#connect Spark to to postgreSQL database
url_db = f"jdbc:postgresql://{CONFIG_DB['host']}:{CONFIG_DB['port']}/{CONFIG_DB['database']}"
properties_dbspark = {
        "user":     f"{CONFIG_DB['user']}",
        "password": f"{CONFIG_DB['password']}",
        "driver":   "org.postgresql.Driver"
    }

#load PySpark DataFrame to postresql database
dfs.write.jdbc(
                url         =   url_db, 
                table       =   TABLE_NAME_TARGET, 
                mode        =   "append", 
                properties  =   properties_dbspark
            )